In [1]:
#!pip install --user Fiona-1.8.6-cp37-cp37m-win_amd64.whl
#!pip install --user Shapely-1.6.4.post2-cp37-cp37m-win_amd64.whl
#!pip install --user GDAL-2.4.1-cp37-cp37m-win_amd64.whl

In [4]:
from libfunc import *

# xosrm.RequestConfig.host = "127.0.0.1:5000"
xosrm.RequestConfig.host = "52.236.141.167:8080"

In [5]:
''' 
Расчет пробега на каждый день недели для точек в заданом порядке
Необходимо указать путь к файлу с расписанием и координаты точки отправления и прибытия
Расписание НЕ ДОЛЖНО содержать ТТ Новый для КПК
Вывод: таблица расстояний
'''
schedule_data_route = open_excel_file(
    file_path='C:\\Users\\andre\\OneDrive\\Рабочий стол\\sav\\Odessa\\Расписания\\Бреславська Валерія Володимирівна.xls')

# Словарь с точками разбитый по интервалам и дням недели
full_dict_route = schedule(schedule_data_route)

data = calc_dist(full_dict_route)
# Отображение таблицы расстояний
display(df_first(data))
display(df_second(data))

,1-ПН,2-ВТ,3-СР,4-ЧТ,5-ПТ
четная нед.,34.4166,111.2974,39.6092,53.6489,56.9389
не четная нед.,34.7119,88.6372,37.2930,59.1302,87.7739


,1-ПН,2-ВТ,3-СР,4-ЧТ,5-ПТ
четная нед.,34.4166,111.2974,39.6092,53.3737,56.9390
не четная нед.,34.7119,88.6372,37.2930,59.1302,87.7739


In [4]:
''' 
Расчет TRIP_ROUTE (Жадный алгоритм)
Расчет пробега на каждый день недели для точек с оптимальной последовательностью
Необходимо указать путь к файлу с расписанием
Расписание ДОЛЖНО содержать Новый для КПК первой точкой на день с минимальным индесом (0 или 1)
Вывод: таблица дистанций
'''    
file_path='C:\\Users\\andre\\OneDrive\\Рабочий стол\\sav\\Тоба пробег.xls'
schedule_data_trip = open_excel_file(file_path)

short_file_name = file_path.split('\\')[-1].split('.')[0]
# Словарь с точками разбитый по интервалам и дням недели
full_dict_trip = schedule(schedule_data_trip)

# вызов функции
data = calc_trips(full_dict_trip)
# Запись  файл excel
write_to_excel(data, short_file_name)
# Отображение таблиц
display(df_first(data[0]))
display(df_second(data[0]))


,1-ПН,2-ВТ,3-СР,4-ЧТ,5-ПТ
четная нед.,93.7298,186.6305,81.3003,188.0819,22.3332
не четная нед.,25.7025,188.5898,86.8914,227.9445,18.9728


,1-ПН,2-ВТ,3-СР,4-ЧТ,5-ПТ
четная нед.,93.7298,185.6036,81.3003,188.0819,22.3332
не четная нед.,25.7025,188.5898,86.8914,227.9445,18.9728


In [2]:
'''
Расчет оптимальных пробегов для всех расписаний в папке
На вход: путь к папке с файлами расписаний
Вывод: таблицы с пробегами по каждому дню
'''
# Путь к папке с расписаниями
path = 'C:\\Users\\andre\\OneDrive\\Рабочий стол\\sav\\Vinnitsa2\\'

# Вызов функции
calc_folder(path)    

Vinnitsa2  -  Глухенький новий
Первая половина месяца:


,1-ПН,2-ВТ,3-СР,4-ЧТ,5-ПТ
четная нед.,186.4578,40.1640,163.9536,183.0658,156.8168
не четная нед.,184.5503,77.4973,158.8398,152.1708,156.9407


Вторая половина месяца:


,1-ПН,2-ВТ,3-СР,4-ЧТ,5-ПТ
четная нед.,183.6966,34.1335,158.4065,183.0658,156.8168
не четная нед.,184.4495,76.9075,158.2065,151.1773,156.9408


Vinnitsa2  -  Денисенко новий
Первая половина месяца:


...

,1-ПН,2-ВТ,3-СР,4-ЧТ,5-ПТ
четная нед.,235.1585,24.4848,182.5644,35.3022,86.1495
не четная нед.,234.1345,24.6795,266.0020,32.8939,94.8157


Vinnitsa2  -  Савінков новий
Первая половина месяца:


,1-ПН,2-ВТ,3-СР,4-ЧТ,5-ПТ
четная нед.,316.3341,43.2969,272.7351,78.9349,150.3086
не четная нед.,326.8264,37.9349,199.4277,92.4894,151.2663


Вторая половина месяца:


,1-ПН,2-ВТ,3-СР,4-ЧТ,5-ПТ
четная нед.,298.1831,34.7447,272.7351,78.9349,150.2709
не четная нед.,326.8264,37.0185,199.4277,92.4894,150.5164


In [3]:
from libfunc import *

# xosrm.RequestConfig.host = "127.0.0.1:5000"
xosrm.RequestConfig.host = "52.236.141.167:8080"

path = 'C:\\Users\\andre\\Downloads\\Telegram Desktop\\тест.xlsx'
data = pd.read_excel(path)

unique_val = data['ФИО ТП'].unique()

writer = pd.ExcelWriter('Расписания ТП расчетные.xlsx', engine = 'xlsxwriter')

empty_table = pd.DataFrame()
empty_table.to_excel(writer, sheet_name = 'Пробег', index=False)

for index, psr in enumerate(unique_val):
    psr_schedule = data[data['ФИО ТП']==psr]
    full_dict_route = schedule(psr_schedule)
    result = calc_trips(full_dict_route)
    write_to_excel2(writer, result, psr, index)
    print(psr + ' FINISHED')
    
worksheet = writer.sheets['Пробег']
for col in range(0, 21, 5):
    worksheet.write(2, 1 + col, 'ПН')
    worksheet.write(2, 2 + col, 'ВТ')
    worksheet.write(2, 3 + col, 'СР')
    worksheet.write(2, 4 + col, 'ЧТ')
    worksheet.write(2, 5 + col, 'ПТ')
    
# Merge 3 cells.
worksheet.merge_range('B1:K1', 'Первая половина месяца')
worksheet.merge_range('L1:U1', 'Вторая половина месяца')
worksheet.merge_range('B2:F2', 'Не четная неделя')
worksheet.merge_range('G2:K2', 'Четная неделя')
worksheet.merge_range('L2:P2', 'Не четная неделя')
worksheet.merge_range('Q2:U2', 'Четная неделя')

writer.save()
writer.close()

print('Файл создан')

Файл создан
